In [44]:
import pandas as pd
import gpxpy
from tqdm import tqdm

In [52]:
def gpx_to_dataframe(gpx_file_path):
    with open(gpx_file_path, 'r', encoding='UTF8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time': point.time,
                })

    df = pd.DataFrame(data)
    return df

In [53]:
gpx_file_path = "./dataset/1.2~2.16/2020-2-16_대모산-구룡산_20200216_101542(ifiam-20200216_134330).gpx"
gpx_dataframe = gpx_to_dataframe(gpx_file_path)
gpx_dataframe

,latitude,longitude,elevation,time
0,37.486959,127.101903,57.936741,2020-02-16 10:15:41+00:00
1,37.486865,127.101911,51.182147,2020-02-16 10:16:10+00:00
2,37.486775,127.101952,51.436948,2020-02-16 10:16:15+00:00
3,37.486696,127.102020,52.436972,2020-02-16 10:16:23+00:00
4,37.486637,127.102109,54.119463,2020-02-16 10:16:33+00:00
...,...,...,...,...
644,37.465106,127.045592,116.845413,2020-02-16 13:41:33+00:00
645,37.465065,127.045490,115.400308,2020-02-16 13:41:43+00:00
646,37.465028,127.045385,114.724567,2020-02-16 13:41:48+00:00
647,37.464963,127.045307,113.530625,2020-02-16 13:41:57+00:00


In [54]:
gpx_file_path = "./대모산_20200216_154011(inhoko22-20200216_173051).gpx"
gpx_dataframe = gpx_to_dataframe(gpx_file_path)
gpx_dataframe

,latitude,longitude,elevation,time
0,37.470078,127.090793,79.259798,2020-01-01 09:41:15+00:00
1,37.470108,127.090585,69.714439,2020-01-01 09:42:29+00:00
2,37.470045,127.090505,71.412487,2020-01-01 09:42:38+00:00
3,37.469993,127.090387,70.843493,2020-01-01 09:42:47+00:00
4,37.469969,127.090263,72.755356,2020-01-01 09:42:53+00:00
...,...,...,...,...
603004,37.483784,127.082562,62.508294,2020-02-16 17:26:25+00:00
603005,37.484481,127.082813,89.371421,2020-02-16 17:28:32+00:00
603006,37.484762,127.082831,67.525633,2020-02-16 17:29:11+00:00
603007,37.484725,127.082683,82.343784,2020-02-16 17:29:14+00:00


In [55]:
gpx_dataframe

,latitude,longitude,elevation,time
0,37.470078,127.090793,79.259798,2020-01-01 09:41:15+00:00
1,37.470108,127.090585,69.714439,2020-01-01 09:42:29+00:00
2,37.470045,127.090505,71.412487,2020-01-01 09:42:38+00:00
3,37.469993,127.090387,70.843493,2020-01-01 09:42:47+00:00
4,37.469969,127.090263,72.755356,2020-01-01 09:42:53+00:00
...,...,...,...,...
603004,37.483784,127.082562,62.508294,2020-02-16 17:26:25+00:00
603005,37.484481,127.082813,89.371421,2020-02-16 17:28:32+00:00
603006,37.484762,127.082831,67.525633,2020-02-16 17:29:11+00:00
603007,37.484725,127.082683,82.343784,2020-02-16 17:29:14+00:00


In [56]:
import folium
import pandas as pd

# gpx_dataframe: latitude, longitude, elevation, time 정보가 담긴 시계열 트랙 데이터프레임

# Create a base map centered at the mean latitude and longitude
center_lat = gpx_dataframe['latitude'].mean()
center_lon = gpx_dataframe['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a GeoJSON FeatureCollection with properties including time
feature_collection = {
    "type": "FeatureCollection",
    "features": []
}

# Define a function to determine marker color based on elevation
def get_marker_color(elevation):
    if elevation < 50:
        return 'green'
    elif elevation < 100:
        return 'orange'
    else:
        return 'red'

# Add custom markers for each data point
for index, row in gpx_dataframe.iterrows():
    marker = folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=get_marker_color(row['elevation']),
        fill=True,
    )
    marker.add_to(m)

for index, row in tqdm(gpx_dataframe.iterrows()):
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']],
        },
        "properties": {
            "time": row['time'].isoformat(),
            "elevation": row['elevation'],
        },
    }
    feature_collection['features'].append(feature)
    

# Save the map to an HTML file
m.save("map.html")

603009it [00:46, 12958.95it/s]


In [18]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd

# gpx_dataframe: latitude, longitude, elevation, time 정보가 담긴 시계열 트랙 데이터프레임

# Create a base map centered at the mean latitude and longitude
center_lat = gpx_dataframe['latitude'].mean()
center_lon = gpx_dataframe['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a GeoJSON FeatureCollection with properties including time
feature_collection = {
    "type": "FeatureCollection",
    "features": []
}

# Define a function to determine marker color based on elevation
def get_marker_color(elevation):
    if elevation < 50:
        return 'green'
    elif elevation < 100:
        return 'orange'
    else:
        return 'red'

# Add custom markers for each data point
for index, row in gpx_dataframe.iterrows():
    marker = folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=get_marker_color(row['elevation']),
        fill=True,
    )
    marker.add_to(m)

for index, row in tqdm(gpx_dataframe.iterrows()):
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']],
        },
        "properties": {
            "time": row['time'].isoformat(),
            "elevation": row['elevation'],
        },
    }
    feature_collection['features'].append(feature)

# Add TimestampedGeoJson plugin to the map with adjusted options
TimestampedGeoJson(
    feature_collection,
    period="PT5S",  # Adjust the time interval as needed
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=1,  # Adjust the speed (lower value is slower)
    date_options="YYYY-MM-DD HH:mm:ss",  # Adjust the date display format
).add_to(m)  # Add the point_to_layer function directly
    

# Save the map to an HTML file
m.save("Entire_track_map.html")

603009it [00:44, 13615.09it/s]
